In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
df_meal

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
import pandas as pd

# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0]
df_grouped_meal = df_meal.groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'

In [ ]:
# Reset the index
df_play = df_meal.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0]
df_grouped_play = df_play.groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_play.index.name = 's_id'

In [ ]:
# Reset the index of the grouped dataframes
df_grouped_meal = df_grouped_meal.reset_index()
df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'

# Concatenate the DataFrames
final_df = pd.concat([df_grouped_meal, df_grouped_play], axis=0)

final_df

In [ ]:
#drop last column from final_df
final_df = final_df.drop(columns=['index'])

In [ ]:
final_df.set_index('s_id', inplace=True)
final_df


In [ ]:
final_df = final_df.drop(columns=['level_0'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
final_df['label'] = final_df['label'].map({0: 'no_rad', 1: 'rad'})

In [ ]:
final_df

In [ ]:


# Perform a grid search for each classifier
X = final_df.drop(['label','group'], axis=1)
y = final_df['label']
groups = final_df['group']

# Create a GroupKFold object
gkf = KFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]


import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
X = final_df.drop(['label','group'], axis=1)
y = final_df['label']
groups = final_df['group']


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = LogisticRegression()

# Initialize the selector
# Fit the model
model.fit(X_train, y_train)

# Get the coefficients of the model
coefficients = model.coef_[0]

# Get the absolute values of the coefficients
abs_coefficients = abs(coefficients)

# Create a DataFrame with the feature names and coefficients
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': abs_coefficients})

# Sort the DataFrame by the coefficients in descending order
feature_importances = feature_importances.sort_values('importance', ascending=False)

# Print the features in order of their importance
print(feature_importances)

In [ ]:
# Fit the model
model.fit(X_train, y_train)

# Get the coefficients of the model
coefficients = model.coef_[0]

# Create a DataFrame with the feature names and coefficients
feature_importances = pd.DataFrame({'feature': X.columns, 'coefficient': coefficients})

# Calculate the absolute values of the coefficients for the importance
feature_importances['importance'] = abs(feature_importances['coefficient'])

# Sort the DataFrame by the importance in descending order
feature_importances = feature_importances.sort_values('importance', ascending=False)

# Print the features in order of their importance, including the sign of the coefficients
print(feature_importances)

In [ ]:
import numpy as np

# Get the sign of the coefficients
feature_importances['coefficient'] = np.sign(feature_importances['coefficient'])

# Replace positive and negative values with '+' and '-'
feature_importances['coefficient'] = feature_importances['coefficient'].map({1: '+', -1: '-'})

In [ ]:
import matplotlib.pyplot as plt

# Assume that 'feature_importances' is a DataFrame where the first column is the feature names
# and the second column is the feature importances

# Get only the features with negative coefficients
negative_features = feature_importances[feature_importances.iloc[:, 1] < 0]

# Sort the DataFrame by the absolute values of the feature importances in descending order
sorted_features = negative_features.sort_values(by=negative_features.columns[1], key=abs, ascending=False)

# Get the 10 most important feature names and absolute importances
important_feature_names = sorted_features.iloc[:10, 0]
important_feature_importances = sorted_features.iloc[:10, 1].abs()

# Plot the feature importances
plt.figure(figsize=(20, 10))  # Increase the figure size to make the labels more readable
plt.title("Feature importances no_rad")
plt.bar(important_feature_names, important_feature_importances, color="b", align="center")
plt.xticks(rotation=45, fontsize = 14)  # Rotate the x-axis labels to make them more readable
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assume that 'feature_importances' is a DataFrame where the first column is the feature names
# and the second column is the feature importances

# Get only the features with positive coefficients
positive_features = feature_importances[feature_importances.iloc[:, 1] > 0]

# Sort the DataFrame by the feature importances in descending order
sorted_features = positive_features.sort_values(by=positive_features.columns[1], ascending=False)

# Get the 10 most important feature names and importances
important_feature_names = sorted_features.iloc[:10, 0]
important_feature_importances = sorted_features.iloc[:10, 1]

# Plot the feature importances
plt.figure(figsize=(20, 10))  # Increase the figure size to make the labels more readable
plt.title("Feature importances rad")
plt.bar(important_feature_names, important_feature_importances, color="b", align="center")
plt.xticks(rotation=45, fontsize=14)  # Rotate the x-axis labels to make them more readable
plt.show()

In [ ]:
feature_importances.to_csv('/Users/andrei-macpro/Documents/Data/tracking/features/combined_feature_importance.csv')

In [ ]:
# Get the 15 most important features
top_15_features = feature_importances.sort_values('importance', ascending=False).head(15)

print(top_15_features)

In [ ]:
import matplotlib.pyplot as plt
# Assuming 'label' is your target variable
class_counts = final_df['label'].value_counts()

print(class_counts)

class_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


#import dummy classifier
from sklearn.dummy import DummyClassifier
# import random state
from numpy.random import RandomState
# Create a dummy classifier
# import make scorer
from sklearn.metrics import make_scorer
#import precision score
from sklearn.metrics import precision_score
#import recall score    
from sklearn.metrics import recall_score
#import f1 score
from sklearn.metrics import f1_score
# import cross validate
from sklearn.model_selection import cross_validate

dummy = DummyClassifier(strategy='most_frequent')

# Create an SVC classifier with specific parameters
svc_rbf = SVC(kernel='rbf', C=1, gamma=0.1)

# Initialize a RandomState
rs = RandomState(123)

# Initialize a GroupKFold
gkf = KFold(n_splits=5)

# Define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average='macro'),
           'recall': make_scorer(recall_score, average='macro'),
           'f1': make_scorer(f1_score, average='macro')}

# Create pipelines
pipeline_dummy = Pipeline([('scaler', StandardScaler()), ('dummy', dummy)])
pipeline_svc = Pipeline([('scaler', StandardScaler()), ('svc', svc_rbf)])

# Perform 10 random shuffles
for i in range(10):
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=rs)

    # Calculate the cross-validated scores of the dummy classifier
    scores_dummy = cross_validate(pipeline_dummy, X_shuffled, y_shuffled, groups=groups_shuffled, cv=gkf, scoring=scoring)

    print(f'Cross-validated scores of dummy classifier for shuffle {i+1}:')
    for metric, scores in scores_dummy.items():
        print(f'{metric}: {scores.mean()}')

    # Calculate the cross-validated scores of the SVC classifier
    scores_svc = cross_validate(pipeline_svc, X_shuffled, y_shuffled, groups=groups_shuffled, cv=gkf, scoring=scoring)

    print(f'Cross-validated scores of SVC classifier for shuffle {i+1}:')
    for metric, scores in scores_svc.items():
        print(f'{metric}: {scores.mean()}')

In [ ]:
final_df.describe()

In [ ]:
import matplotlib.pyplot as plt
final_df.hist(bins=30, figsize=(20,15))
plt.show()

In [ ]:
sns.pairplot(final_df)
plt.show()